In [6]:
%load_ext autoreload
%autoreload 2
import energyderivatives as ed

In [7]:
import matplotlib.pyplot as plt

In [8]:
import numpy as np

In [9]:
S = 100
K = 100
t = 1 / 12
sigma = 0.4
r = 0.10
b = 0.1

path_length = 30
mc_paths = 500000
mc_loops = 1

In [10]:
opt = ed.GBSOption(S, K, t, r, b, sigma)

In [11]:
inno = ed.monte_carlo_options.NormalSobolInnovations
path = ed.monte_carlo_options.WienerPath
payoff = ed.monte_carlo_options.PlainVanillaPayoff

In [15]:
eps = inno(mc_paths, path_length).sample_innovation()

In [17]:
pp = path(eps, sigma, t/path_length, b)

In [21]:
payoff(pp, S, K, t, r, b, sigma).call().shape

(500000,)

In [67]:
mc = ed.monte_carlo_options.MonteCarloOption(
    mc_loops, path_length, mc_paths, 
    S, K, t, r, b, sigma,
    inno, path, payoff,
    trace=False, antithetic=True, standardization=False
)


In [68]:
mc.call().mean()

5.012862753307121

In [69]:
opt.call()

5.011784286195763

In [58]:
mc.put().mean()

4.177224500256043